<!-- @format -->

#### Do all the imports


In [ ]:
import requests
from dotenv import load_dotenv
from os import getenv
from json import dump
import pickle

<!-- @format -->

#### load the environments


In [ ]:
load_dotenv("techtalk_SEC.secrets")
load_dotenv("techtalk_SEC.env")

<!-- @format -->

#### prepare the request


In [ ]:
r = requests.Request(
    method="GET",
    url=f'{getenv("WEB_URL")}/files/company_tickers.json',
    headers={
        "User-Agent": "PWTTechTalkBot/1.0 Port Wallis Technologies contact - rwolfe@portwallistechnologies.com",
        "Accept": "application/json",
    },
)

s = requests.Session()

<!-- @format -->

#### Go get the data


In [ ]:
req = s.send(r.prepare())

<!-- @format -->

#### Save the response to an external file (completely optional)


In [ ]:
with open(file="cik_tickers.json", mode="w") as file:
    dump(req.json(), file)

#### stash the ticker mapping into a dict for later


In [ ]:
the_json = req.json()
ticker_cik = {the_json[a]["ticker"]: str(the_json[a]["cik_str"]).zfill(10) for a in the_json}
cik_ticker = {str(the_json[a]["cik_str"]).zfill(10): the_json[a]["ticker"] for a in the_json}

<!-- @format -->

#### Pull your favorite company's CIK from the dictionary 

In [ ]:
ticker_symbol = "IBM"

company_cik = ticker_cik[ticker_symbol]

<!-- @format -->

#### And go get the list of "concepts" submissions for that company


In [ ]:
r.url = f"""{getenv("DATA_URL")}/api/xbrl/companyfacts/CIK{company_cik}.json""" 

req = s.send(r.prepare())

<!-- @format -->

#### Save the results to a file again


In [ ]:
with open(file=f"company-facts-{ticker_symbol}.json", mode="w") as file:
    dump(req.json(), file)



#### save all of the facts to a couple of dicts for later use


In [ ]:
company_facts: dict = req.json()
print (len(pickle.dumps(company_facts)))
a = sorted([f for  f in company_facts["facts"]["us-gaap"] if  "Revenues" == f or "NetIncomeLoss" == f] )


#### Go get the revene and income information for the company





In [ ]:
r.url = f"""{getenv("DATA_URL")}/api/xbrl/companyconcept/CIK{company_cik}/gaap/Revenues.json""" 
req = s.send(r.prepare())

the_json = req.json()